In [72]:
import pandas as pd

# Perfoming NLP on input

In [32]:
pip install spacy

  Using cached spacy_legacy-3.0.5-py2.py3-none-any.whl (12 kB)
  Using cached typer-0.3.2-py3-none-any.whl (21 kB)
  Using cached pathy-0.5.2-py3-none-any.whl (42 kB)
  Using cached catalogue-2.0.4-py3-none-any.whl (16 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached smart_open-3.0.0.tar.gz (113 kB)
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107101 sha256=c19ee98d400c2d5a99be20cd032527a8eb5f80375c3f23983534ac418f65bce4
  Stored in directory: c:\users\anurag\appdata\local\pip\cache\wheels\11\73\9a\f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [34]:
!python -m spacy download en_core_web_lg


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [33]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [175]:
from pathlib import Path
import spacy
import en_core_web_sm

nlp1 = spacy.load('en_core_web_lg')

output_dir=Path(r"C:\Users\Anurag\Desktop\NLP_model_for_NER\ner")
nlp2 = spacy.load(output_dir)
Trip_type = []
doc = nlp2("I want an adventurous trip")
for ent in doc.ents:
        Trip_type.append(ent.text.capitalize())
print("Trip_type:" ,Trip_type)

Trip_type: ['Adventurous']


In [176]:
Trip_type

['Adventurous']

# Function to get recommended cities

In [177]:
POI_TripTypes_count = pd.read_csv("POI_TripTypes_count.csv")

def recc_city(trip_type_list):
    recc_city_df = (POI_TripTypes_count.loc[(POI_TripTypes_count["Trip_type"].isin(trip_type_list))]).groupby(["Destination_name"]).sum().nlargest(3,"Count")
    return list(recc_city_df.index)

In [178]:
#POI_TripTypes_count

In [181]:
#nlp = ['Adventurous', 'Activites'] #This list will be output of NLP performed on user input.
rec_cities = recc_city(Trip_type)
print(rec_cities) #This is list of recommended cities for user to choose from and will be sent as output for user.

['Anaheim', 'Lake Tahoe', 'Palm Springs']


# Taking input city from user

In [182]:
POI_unique = pd.read_csv("POI_unique.csv") 
#POI_unique

In [183]:
#days = number of days entered by user for trip
#rec_city = city chosen by user
#nlp = list of trip types extracted from user input

#This function will reture POIs in form of dataframe
def get_recc_pois(days, rec_city, nlp):
    flag = False
    filteredData0 = POI_unique.loc[(POI_unique['Destination_name'] == rec_city) & (POI_unique['Trip_type'] == nlp[0])]
    filteredData0 = filteredData0.sort_values(by=['POI_reviews_count','Average_Rating'], ascending=False)


    if len(nlp) == 2:
        
        flag = True
        filteredData1 = POI_unique.loc[(POI_unique['Destination_name'] == rec_city) & (POI_unique['Trip_type'] == nlp[1])]
        filteredData1 = filteredData1.sort_values(by=['POI_reviews_count','Average_Rating'], ascending=False)
        #print(nlp[1])

    if flag:
        if days % 2 == 0:
            Trip_type_1 = filteredData0.head(int(days/2))
            Trip_type_2 = filteredData1.head(int(days/2))
        else:
            if len(filteredData0["POI_name"]) > len(filteredData1["POI_name"]):
                Trip_type_1 = filteredData0.head(int((days+1)/2))
                Trip_type_2 = filteredData1.head(days//2)
            else:
                Trip_type_1 = filteredData0.head(days//2)
                Trip_type_2 = filteredData1.head(int((days+1)/2))
        POI_final = pd.concat([Trip_type_1, Trip_type_2])
    else:
        POI_final = filteredData0.head(days)
    
    #print(nlp)
    return POI_final

In [184]:
Trip_type

['Adventurous']

In [191]:
#Dummy input
days = 3
rec_city = "Lake Tahoe"
#nlp = ["Historic", "Relaxed"] 

POI_final = get_recc_pois(days, rec_cities[2], Trip_type)
POI_final

,POI_name,Average_Rating,Destination_name,POI_reviews_count,POI_category,POI_description,POI_rating,POI_address,POI_Suggested_Duration,POI_open_close_hours,Review_Sentiment,Trip_type
78,Coachella Valley Preserve,4.5,Palm Springs,726,Nature & Wildlife Areas,"Formed by a system of sand dunes, this 20,000-...",4.5,"29200 Thousand Palms Canyon Road, Palm Springs...",NaN,Hours\nSun - Sat\n7:00 AM - 5:00 PM,Positive,Adventurous
158,Indian Canyons Hiking Trails,4.5,Palm Springs,314,Hiking Trails,NaN,4.5,"385 Stanford Drive, Palm Springs, Greater Palm...",NaN,NaN,Positive,Adventurous
388,South Lykken Trail,4.5,Palm Springs,149,Nature & Wildlife Areas,The creosote bush is one of the interesting de...,4.5,"El Estero, Palm Springs, Greater Palm Springs, CA",NaN,NaN,Positive,Adventurous


# Extracting nearby POIs

In [192]:
#Loading nearby data
nearby = pd.read_csv("final_nearby.csv")
#nearby

In [193]:
#List of POIs from recommended POIs
pois_list = list(POI_final["POI_name"])


def get_nearby_pois(pois_list):
    #cols = ["POI_name","POI_nearby_name"]
    all_points = []
    #nearby_points = pd.DataFrame(columns = cols)
    for points in pois_list:
        all_points.append(points)
        fn = nearby.loc[(nearby['POI_name'] == points)]
        fn = fn.sort_values(by=['Distance_miles'], ascending=True)
        fn = fn.head(3)
        for near in fn["POI_nearby_name"].unique():
            all_points.append(near.replace("'",""))
        #nearby_points = pd.concat([nearby_points, fn.head(3)])
    
    return all_points
   
get_nearby_pois(pois_list) #returns a dataframe containing three nearby points for every POI

['Coachella Valley Preserve',
 'Indian Canyons Hiking Trails',
 'South Lykken Trail']

In [150]:
pois_for_itinerary

['Disneyland Park',
 'Tarzans Treehouse',
 'The Disney Gallery',
 'Star Wars: Galaxy’s Edge',
 'Disney California Adventure Park',
 'Best Coast Tours - Orange County',
 'Anaheim Convention Center',
 'Anaheim Packing District',
 'Anaheim Convention Center',
 'Fireworks at Disneyland Park',
 'Goofys Playhouse',
 'Disney California Adventure Park']

In [164]:
pois_for_itinerary["POI_nearby_name"].unique()[0:3]

array(['Muscle Beach Venice', 'Venice art walls', 'High Rooftop Lounge'],
      dtype=object)

In [100]:
filteredData1 = POI_unique.loc[(POI_unique['Destination_name'] == rec_city) & (POI_unique['Trip_type'] == "Activities")]

In [101]:
filteredData1

,POI_name,Average_Rating,Destination_name,POI_reviews_count,POI_category,POI_description,POI_rating,POI_address,POI_Suggested_Duration,POI_open_close_hours,Review_Sentiment,Trip_type
12,Anaheim Convention Center,4.5,Anaheim,968,Conference & Convention Centers,NaN,4.5,"800 W Katella Ave, Anaheim, CA 92802-3496",NaN,NaN,Positive,Activities
40,Bibbidi Bobbidi Boutique,4.5,Anaheim,67,Character Experiences,NaN,4.5,"1313 Disneyland Drive Fantasyland, Anaheim, CA...",NaN,NaN,Positive,Activities
